# Text Normalization (Chinese)

- `text_normalizer_zh.py`
- Including functions for:
    - word-seg chinese texts
    - clean up texts by removing duplicate spaces and line breaks
    - remove incompatible weird characters

In [1]:
import unicodedata
import re
#from nltk.corpus import wordnet
#import collections
from nltk.tokenize.toktok import ToktokTokenizer
from bs4 import BeautifulSoup
import requests
import pandas as pd
import text_normalizer_zh as tnz

## Normalization Functions

In [2]:
# %load text_normalizer_zh.py
"""

Notes
-----

    These functions are based on the text normalization functions 
    provided in Text Analytics with Python 2ed.

"""

import unicodedata
import re
# from nltk.tokenize.toktok import ToktokTokenizer
import pandas as pd
import jieba

## Initialize Trad Chinese dictionary
jieba.set_dictionary('../../../RepositoryData/data/jiaba/dict.txt.jiebatw.txt')


## Normalize unicode characters
def remove_weird_chars(text):
    #     ```
    #     (NFKD) will apply the compatibility decomposition, i.e.
    #     replace all compatibility characters with their equivalents.
    #     ```
    text = unicodedata.normalize('NFKD', text).encode('utf-8',
                                                      'ignore').decode(
                                                          'utf-8', 'ignore')
    return text


## Remove extra linebreaks
def remove_extra_linebreaks(text):
    lines = text.split(r'\n+')
    return '\n'.join(
        [re.sub(r'[\s]+', ' ', l).strip() for l in lines if len(l) != 0])


## Remove extra medial/trailing/leading spaces
def remove_extra_spaces(text):
    return re.sub("\\s+", " ", text).strip()


## Seg the text into words
def seg(text):
    text_seg = jieba.cut(text)
    out = ' '.join(text_seg)
    return out


## Remove punctuation/symbols
def remove_symbols(text):
    """
    
    Unicode 6.0 has 7 character categories, and each category has subcategories:

    Letter (L): lowercase (Ll), modifier (Lm), titlecase (Lt), uppercase (Lu), other (Lo)
    Mark (M): spacing combining (Mc), enclosing (Me), non-spacing (Mn)
    Number (N): decimal digit (Nd), letter (Nl), other (No)
    Punctuation (P): connector (Pc), dash (Pd), initial quote (Pi), final quote (Pf), open (Ps), close (Pe), other (Po)
    Symbol (S): currency (Sc), modifier (Sk), math (Sm), other (So)
    Separator (Z): line (Zl), paragraph (Zp), space (Zs)
    Other (C): control (Cc), format (Cf), not assigned (Cn), private use (Co), surrogate (Cs)
    
    
    There are 3 ranges reserved for private use (Co subcategory): 
    U+E000—U+F8FF (6,400 code points), U+F0000—U+FFFFD (65,534) and U+100000—U+10FFFD (65,534). 
    Surrogates (Cs subcategory) use the range U+D800—U+DFFF (2,048 code points).
    
    
    """

    ## Brute-force version: list all possible unicode ranges, but this list is not complete.
    #   text = re.sub('[\u0021-\u002f\u003a-\u0040\u005b-\u0060\u007b-\u007e\u00a1-\u00bf\u2000-\u206f\u2013-\u204a\u20a0-\u20bf\u2100-\u214f\u2150-\u218b\u2190-\u21ff\u2200-\u22ff\u2300-\u23ff\u2460-\u24ff\u2500-\u257f\u2580-\u259f\u25a0-\u25ff\u2600-\u26ff\u2e00-\u2e7f\u3000-\u303f\ufe50-\ufe6f\ufe30-\ufe4f\ufe10-\ufe1f\uff00-\uffef─◆╱]+','',text)

    text = ''.join(ch for ch in text
                   if unicodedata.category(ch)[0] not in ['P', 'S'])
    return text


## Remove numbers
def remove_numbers(text):
    return re.sub('\\d+', "", text)


## Remove alphabets
def remove_alphabets(text):
    return re.sub('[a-zA-Z]+', '', text)


## Combine every step
def normalize_corpus(corpus,
                     is_remove_extra_linebreaks=True,
                     is_remove_weird_chars=True,
                     is_seg=True,
                     is_remove_symbols=True,
                     is_remove_numbers=True,
                     is_remove_alphabets=True):

    normalized_corpus = []
    # normalize each document in the corpus
    for doc in corpus:

        if is_remove_extra_linebreaks:
            doc = remove_extra_linebreaks(doc)

        if is_remove_weird_chars:
            doc = remove_weird_chars(doc)

        if is_seg:
            doc = seg(doc)

        if is_remove_symbols:
            doc = remove_symbols(doc)

        if is_remove_alphabets:
            doc = remove_alphabets(doc)

        if is_remove_numbers:
            doc = remove_numbers(doc)

        normalized_corpus.append(remove_extra_spaces(doc))

    return normalized_corpus

## Extract an Article

- Grab the first article from Google news for demonstration

In [3]:
url = 'https://news.google.com/topics/CAAqJQgKIh9DQkFTRVFvSUwyMHZNRFptTXpJU0JYcG9MVlJYS0FBUAE?hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant'
r = requests.get(url)
web_content = r.text
soup = BeautifulSoup(web_content, 'lxml')
title = soup.find_all('a', class_='DY5T1d')
first_art_link = title[0]['href'].replace('.', 'https://news.google.com', 1)

#print(first_art_link)
art_request = requests.get(first_art_link)
art_request.encoding = 'utf8'
soup_art = BeautifulSoup(art_request.text, 'lxml')

art_content = soup_art.find_all('p')
art_texts = [p.text for p in art_content]
print(art_texts)

['萊克多巴胺美豬明年進口台灣，台中等多個縣市食安自治條例規定萊克多巴胺零檢出，也有其他縣市打算比照辦理，但衛福部卻發文指出與母法抵觸，應配合修正；行政院函文則表示有抵觸中央法規疑慮，應提出意見說明。衛福部表示，正在重新簽辦發文，內容將與行政院一致。', '\n', '\r\n立法院今天審查農委會101年9月7日農防字第1011473960號公告，多名立委要求衛福部說明地方萊克多巴胺零檢出的自治條例跟中央法規抵觸問題。衛福部次長薛瑞元原本指出，中央訂定的萊客多巴胺殘留容許量從明年1月1日開始生效，因此地方政府現在若訂定零檢出仍為有效，但要開始檢討，否則到明年將會抵觸母法而無效。', '\n', '\r\n根據衛福部9月30日發給地方政府的函，明確指出動物用藥殘留標準已經過中央發布，「自治法規如有與該標準抵觸情形，應配合修正」，並說明動物用藥殘留標準所定殘留容許量屬食品安全衛生標準，具全國一致性質，自治法規不得為較嚴格或寬鬆之規定。', '\n', '\r\n不過行政院10月26日發給台中市府函文態度則較保守，僅表示地方萊克多巴胺的零檢出自治條例有抵制中央法規疑慮，應於一個月內提出意見說明。', '\n', '\r\n立委林奕華質疑，衛福部發文給地方是直接要求修正自治條例，跟中央說法有所不同，且釋字738號早就說明中央法令是最低標準，地方可以訂定較嚴格標準，且地方自治條例經過議會三讀通過，根本沒問題。', '\n', '\r\n薛瑞元表示，之前是請地方檢討，不過現在正在重新簽辦發文，內文將與行政院一樣。至於釋字738號是說地方可以因地制宜規定，但既然中央的食安法已經有規定，就應該遵守。', '\n                    政府開放進口萊克多巴胺美豬，有十一縣市擬訂定自治條例要求萊劑零檢出，本報昨天報導地方政府近日收到行政院和衛福部函文，行政...                  ', '\n                    行政院函文要求地方修改萊劑零檢出的食安自治條例，台中市長盧秀燕、新北市長侯友宜等藍營縣市首長重申堅持零檢出，台北市長柯文...                  ', '\n                    民進黨立委林淑芬日前針對萊豬議題提案修正「學校衛生法」，明定學校禁用萊豬及其加工品，該案將在周五院會闖關。但傳出在

- Normalized results:

In [4]:
tnz.normalize_corpus([' '.join(art_texts)])

Building prefix dict from /Users/Alvin/GoogleDrive/_MySyncDrive/RepositoryData/data/jiaba/dict.txt.jiebatw.txt ...


Loading model from cache /var/folders/n7/ltpzwx813c599nfxfb94s_640000gn/T/jieba.u1b52b47246a0f2e6497af6bbe107adac.cache


Loading model cost 0.547 seconds.


Prefix dict has been built successfully.


['萊克 多巴胺 美豬 明年 進口 台灣 台中 等 多個 縣市 食安 自治 條例 規定 萊克 多巴胺 零 檢出 也 有 其 他 縣市 打算 比照 辦理 但 衛福部 卻 發文 指出 與 母法 抵觸 應 配合 修正 行政院 函文 則 表示 有 抵觸 中央 法規 疑慮 應 提出 意見 說明 衛福部 表示 正在 重新 簽辦 發文 內容 將與 行政院 一致 立法院 今天 審查 農委會 年 月 日 農防字 第 號 公告 多名 立委 要求 衛福部 說明 地方 萊克 多巴胺 零 檢出 的 自治 條例 跟 中央 法規 抵觸 問題 衛福部 次長 薛瑞元 原本 指出 中央 訂定 的 萊客 多巴胺 殘留 容許量 從 明年 月 日 開始 生效 因此 地方 政府 現在 若 訂定 零 檢出 仍為 有效 但 要 開始 檢討 否則 到 明年 將會 抵觸 母法 而 無效 根據 衛福部 月 日 發給 地方 政府 的 函 明確 指出 動物 用藥 殘留 標準 已 經過 中央 發布 自治 法規 如有 與 該 標準 抵觸 情形 應 配合 修正 並 說明 動物 用藥 殘留 標準 所定 殘留 容許量 屬 食品 安全 衛生 標準 具 全國 一致 性質 自治 法規 不得 為 較 嚴格 或 寬鬆 之 規定 不過 行政院 月 日 發給 台中 市府 函文 態度 則較 保守 僅 表示 地方 萊克 多巴胺 的 零 檢出 自治 條例 有 抵制 中央 法規 疑慮 應 於 一個 月 內 提出 意見 說明 立委 林奕華 質疑 衛福部 發文 給 地方 是 直接 要求 修正 自治 條例 跟 中央 說法 有所 不同 且 釋字 號早 就 說明 中央 法令 是 最 低 標準 地方 可以 訂定 較 嚴格 標準 且 地方 自治 條例 經過 議會 三讀通過 根本 沒 問題 薛瑞元 表示 之前 是 請 地方 檢討 不過 現在 正在 重新 簽辦 發文 內文 將與 行政院 一樣 至於 釋字 號是 說 地方 可以 因地制宜 規定 但 既然 中央 的 食安法 已經 有 規定 就 應該 遵守 政府 開放 進口 萊克 多巴胺 美豬 有十一 縣市 擬 訂定 自治 條例 要求 萊劑 零 檢出 本報 昨天 報導 地方 政府 近日 收到 行政院 和 衛福部 函文 行政 行政院 函文 要求 地方 修改 萊劑 零 檢出 的 食安 自治 條例 台中 市長 盧秀燕 新北 市長 侯友